In [6]:
import pandas as pd
import numpy as np

In [2]:
dataset_folder = "../all"
!ls {dataset_folder}

data_note.pdf	       test_set_metadata.csv  training_set_metadata.csv
sample_submission.csv  test_set_sample.csv
test_set.csv	       training_set.csv


In [21]:
train = pd.read_csv(f"{dataset_folder}/training_set.csv")
train.head(5)

,object_id,mjd,passband,flux,flux_err,detected
0,615,59750.4229,2,-544.810303,3.622952,1
1,615,59750.4306,1,-816.434326,5.553370,1
2,615,59750.4383,3,-471.385529,3.801213,1
3,615,59750.4450,4,-388.984985,11.395031,1
4,615,59752.4070,2,-681.858887,4.041204,1


In [25]:
maxMjd = 1000

In [ ]:
def mjdToPredict(mjdToExclude, minMjd, maxMjd, step):
    return np.array(list(set(range(minMjd, maxMjd+1, step)) - set(mjdToExclude)))

In [97]:
def fill_missing_flux(objectDf, maxMjd, step):
    objectDf.mjd = objectDf[["mjd"]].astype(np.int64)
    objectDf.mjd = objectDf.mjd - objectDf.mjd.min()
    instance = objectDf.groupby(["passband", "mjd"]).mean().reset_index()
    meanByPassband = instance.groupby("passband").mean()[["flux", "flux_err"]]
    frames = [instance]
    for p in range(6):
        toExclude = instance[instance.passband==p].mjd
        toPredict = mjdToPredict(toExclude, 0, maxMjd, step)
        nb = len(toPredict)
        frames.append(pd.DataFrame({
            'mjd': toPredict,
            'flux': [meanByPassband.loc[p].flux]*nb,
            'flux_err': [meanByPassband.loc[p].flux_err]*nb,
            'detected': [-1] * nb,
            'passband': [p]*nb,
            'object_id': [objectId]*nb
        }))
    return pd.concat(frames, sort=False)

In [100]:
objectId = 615
step = 1
objectDf = train[train.object_id==objectId]
objectDf.passband.value_counts()

0    63
4    58
3    58
2    58
1    58
5    57
Name: passband, dtype: int64

In [101]:
out = fill_missing_flux(objectDf, maxMjd, step)
out.passband.value_counts()

3    1001
2    1001
5    1001
1    1001
4    1001
0    1001
Name: passband, dtype: int64